In [2]:
import arcpy
from arcpy import env
from arcpy.da import *
from arcpy.sa import *
import os
import shutil

In [3]:
arcpy.env.overwriteOutput = True

In [4]:
# 按掩膜提取
def mask_extraction(in_raster,mask_ele,pro_ele):
    ele_pro = arcpy.Describe(pro_ele).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=ele_pro,snapRaster=pro_ele, cellSize=5):
        out_raster = arcpy.sa.ExtractByMask(in_raster, mask_ele, "INSIDE", ele_pro)
        return out_raster

In [5]:
# raster_path = r"D:\ArcGISProjects\workspace\shbyq\tif_file_texture\dy\one_bands"
raster_path = r"D:\ArcGISProjects\workspace\shbyq\features_data_climate.gdb"
mask_ele = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb\DEM"
pro_ele = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb\DEM"
save_path = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb"

In [6]:
env.workspace = raster_path
raster_list = arcpy.ListRasters()
# raster_list.remove('dem.tif')
# raster_list.remove('DEM')
print(raster_list),len(raster_list)

['PRE', 'SRA', 'TMP', 'VAP', 'WIND', 'BIO']


(None, 6)

In [7]:
# 多个掩膜
for one_raster in raster_list:
    result_raster = mask_extraction(one_raster,mask_ele,pro_ele)
    name = str(one_raster).replace(".tif","").replace(" ","").replace("-","")
    result_raster.save(os.path.join(save_path,f"{name}"))
    print(name)

PRE
SRA
TMP
VAP
WIND
BIO


In [12]:
# 单个掩膜
raster_path = r"D:\ArcGISProjects\workspace\shbyq\dy_bands14_pca.tif"
mask_ele = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb\DEM"
pro_ele = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb\DEM"
save_path = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb"

In [10]:
env.workspace = save_path

In [13]:
result_raster = mask_extraction(raster_path,mask_ele,pro_ele)
result_raster.save("PCA")